In [12]:
!curl -L -o fs_inventory4_inventory_NFXHMZLOORXXE6I_snappy_parquet.tar https://www.dropbox.com/s/0a5vxludy7wak8y/fs_inventory4_inventory_nfxhmzloorxxe6i_snappy_parquet.tar.gz?dl=0

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1034    0  1034    0     0   1171      0 --:--:-- --:--:-- --:--:--  1171
100  101M  100  101M    0     0  26.2M      0  0:00:03  0:00:03 --:--:-- 41.6M


In [ ]:
!tar -xzvf fs_inventory4_inventory_NFXHMZLOORXXE6I_snappy_parquet.zip
!mkdir -p /mnt/conduit_parquet_store
!mv fs_inventory4_inventory_NFXHMZLOORXXE6I_snappy_parquet /mnt/conduit_parquet_store/

In [ ]:
#for HADOOP you have to do
!hadoop fs -mkdir /mnt
!hadoop fs -mkdir /mnt/conduit_parquet_store
!hadoop fs -copyFromLocal -r fs_inventory4_inventory_NFXHMZLOORXXE6I_snappy_parquet /mnt/conduit_parquet_store/

In [7]:
import os
import sys
import psutil
import time
import cudf
from blazingsql import BlazingContext
from dask.distributed import Client
import dask_cudf


In [8]:
LOCAL_FOLDER="/mnt/conduit_parquet_store/fs_inventory4_inventory_NFXHMZLOORXXE6I_snappy_parquet"
HDFS_FOLDER="hdfs://172.31.3.117:9000/mnt/conduit_parquet_store/fs_inventory4_inventory_NFXHMZLOORXXE6I_snappy_parquet"


In [6]:
client = Client('172.31.3.117:18085')
bc = BlazingContext(dask_client=client, network_interface='ens5',allocator="default",pool=False)

BlazingContext ready


In [9]:
#this one crashes
hdfs_dask_df = dask_cudf.read_parquet(HDFS_FOLDER)
hdfs_dask_df=bc.unify_partitions(hdfs_dask_df)
bc.create_table("inv",hdfs_dask_df)

ERROR: In getNodePartitions, woker has no corresponding partition
ERROR: In getNodePartitions, woker has no corresponding partition


In [10]:
#error in one of the dask workers
"""
distributed.worker - WARNING -  Compute Failed
Function:  read_parquet_part
args:      (<function CudfEngine.read_partition at 0x7f1e688dc440>, <fsspec.implementations.hdfs.PyArrowHDFS object at 0x7f1e54606910>, Empty DataFrame
Columns: [week, sales_depot_id, sales_channel_id, route_id, client_id, product_id, sales_week_units, sales_value, return_units_next_week, returns_value_next_week, demand, feat_sales_depot_id_cnt_route_id, feat_sales_depot_id_cnt_client_id, feat_sales_depot_id_cnt_product_id, feat_route_id_cnt_sales_depot_id, feat_route_id_cnt_client_id, feat_route_id_cnt_product_id, feat_client_id_cnt_product_id, feat_product_id_cnt_sales_depot_id, feat_product_id_cnt_route_id, forecast, forecast_error_0, bin, forecast_error_1, forecast_error_2, forecast_error_3, forecast_error_4, forecast_error_5, forecast_error_6, forecast_error_7, forecast_error_8, forecast_error_9, forecast_error_10, __index_level_0__]
Index: [], [('/mnt/conduit_parquet_store/fs_inventory4_inventory_NFXHMZLOORXXE6I_snappy_parquet/part-00000-6cc63193-1da6-4475-9dcb-dd2106329cb3-c000.snappy.parq
kwargs:    {}
Exception: KeyError('week')

distributed.worker - WARNING -  Compute Failed
Function:  read_parquet_part
args:      (<function CudfEngine.read_partition at 0x7f1e688dc440>, <fsspec.implementations.hdfs.PyArrowHDFS object at 0x7f1e54606910>, Empty DataFrame
Columns: [week, sales_depot_id, sales_channel_id, route_id, client_id, product_id, sales_week_units, sales_value, return_units_next_week, returns_value_next_week, demand, feat_sales_depot_id_cnt_route_id, feat_sales_depot_id_cnt_client_id, feat_sales_depot_id_cnt_product_id, feat_route_id_cnt_sales_depot_id, feat_route_id_cnt_client_id, feat_route_id_cnt_product_id, feat_client_id_cnt_product_id, feat_product_id_cnt_sales_depot_id, feat_product_id_cnt_route_id, forecast, forecast_error_0, bin, forecast_error_1, forecast_error_2, forecast_error_3, forecast_error_4, forecast_error_5, forecast_error_6, forecast_error_7, forecast_error_8, forecast_error_9, forecast_error_10, __index_level_0__]
Index: [], [('/mnt/conduit_parquet_store/fs_inventory4_inventory_NFXHMZLOORXXE6I_snappy_parquet/part-00000-6cc63193-1da6-4475-9dcb-dd2106329cb3-c000.snappy.parq
kwargs:    {}
Exception: KeyError('week')
"""

"\ndistributed.worker - WARNING -  Compute Failed\nFunction:  read_parquet_part\nargs:      (<function CudfEngine.read_partition at 0x7f1e688dc440>, <fsspec.implementations.hdfs.PyArrowHDFS object at 0x7f1e54606910>, Empty DataFrame\nColumns: [week, sales_depot_id, sales_channel_id, route_id, client_id, product_id, sales_week_units, sales_value, return_units_next_week, returns_value_next_week, demand, feat_sales_depot_id_cnt_route_id, feat_sales_depot_id_cnt_client_id, feat_sales_depot_id_cnt_product_id, feat_route_id_cnt_sales_depot_id, feat_route_id_cnt_client_id, feat_route_id_cnt_product_id, feat_client_id_cnt_product_id, feat_product_id_cnt_sales_depot_id, feat_product_id_cnt_route_id, forecast, forecast_error_0, bin, forecast_error_1, forecast_error_2, forecast_error_3, forecast_error_4, forecast_error_5, forecast_error_6, forecast_error_7, forecast_error_8, forecast_error_9, forecast_error_10, __index_level_0__]\nIndex: [], [('/mnt/conduit_parquet_store/fs_inventory4_inventory_N

In [12]:
#this one executed alone also crashes:
import dask_cudf
hdfs_dask_df = dask_cudf.read_parquet(HDFS_FOLDER)
hdfs_dask_df.compute()


KeyError: 'week'

In [11]:
#this one works
def retrieve_parquet_files_from_folder(folder_name):
    target_files = []
    for entry in os.listdir(folder_name):
        if entry.endswith("parquet"):
            target_file = os.path.join(folder_name, entry)
            target_files.append(target_file)
    return target_files

parquet_files = retrieve_parquet_files_from_folder(LOCAL_FOLDER)
bc = BlazingContext(allocator="default",pool=False)
bc.create_table("inv",parquet_files)
res = bc.sql("select * from inv")

bc.create_table("inv2",res)
bc.sql("select count(*) from inv2")

BlazingContext ready


,COUNT(*)
0,1000000
